In [12]:
anno_dir = "/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/corr_ASE_m6A/oppo/"
os.chdir(anno_dir)
result_dir="/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/corr_ASE_m6A/oppo/for_experiment/"
os.system("mkdir -p %s" % result_dir)

0

In [2]:
df=pd.read_table("/home/galaxy/project/alleleSpecific_analysis/data/hg19_genome/dbsnp_138.hg19.txt",sep=" ",header=None,comment="#")
df.columns = ['chr','pos','rs','ref','alt']
df['term'] = df['chr'] + ":" + df['pos'].astype(str)
df['ref2alt'] = df['ref'] + ">" + df['alt'] + ";" + df['rs']
snp_dict = dict(zip(df['term'], df['ref2alt']))

In [13]:
anno_list = glob.glob("*.txt")
for ind in anno_list:
    array_list = []
    res_file = os.path.join(result_dir, ind.replace(".txt", "_snp.txt"))
    df = pd.read_table(ind)
#     print(df.head())
    for i, values in df.iterrows():
        if values['term'] in snp_dict:
            array_list.append(values.tolist() + [snp_dict[values['term']]])
    df_res = pd.DataFrame(array_list)
    df_res.columns = ['term','m6a_mark','m6a_difference','expression_mark','expression_difference','SNP_info']
    df_res.to_csv(res_file, sep="\t", index=False)

In [14]:
#txt to bed
os.chdir("/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/corr_ASE_m6A/oppo/for_experiment/")
# txt_list = glob.glob("*.txt")
# for txt in txt_list:
#     # term    ASE_type        Allelic-m6A_type        SNP_info
#     df = pd.read_table(txt)
#     df['chr'] = df['term'].str.split(":").str[0]
#     df['pos'] = df['term'].str.split(":").str[1]
#     df['info']=df['m6a_mark']+":"+df['m6a_difference'].astype(str)+":"+df['expression_mark']+":"+ df['expression_difference'].astype(str)+":"+df['SNP_info']
#     df[['chr','pos','pos','info']].to_csv(txt.replace(".txt", '.bed'), sep="\t", header=False, index=False)
    
# bed_list = glob.glob("*.bed")
# gene="/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/readCount/by_ind/sig/snp2gene/gencode.v19.annotation_Gene.bed"
# result_dir = "overlap/"
# for bed in bed_list:
#     res = os.path.join(result_dir, os.path.basename(bed))
#     os.system("bedtools intersect -a %s -b %s -wa -wb > %s" % (bed, gene, res))
    
result_dir = "30nt_bed"
bed_list = glob.glob("*.bed")
for bed in bed_list:
    res = os.path.join(result_dir, bed)
    df = pd.read_table(bed, header=None)
    df.columns = ['chr','s','e','info']
    df['new_s'] = df['s'] - 31 #
    df['new_e'] = df['e'] + 30 #
    df['name'] = df['chr']+":"+df['e'].astype(str)+":"+df['info']
    df[['chr','new_s','new_e','name']].to_csv(res, sep="\t", header=False, index=False)
    
result_dir = "fasta/"
bed_list = glob.glob("30nt_bed/*.bed")
print(len(bed_list))
genome = "/home/galaxy/project/alleleSpecific_analysis/data/hg19_genome/hg19.fa"
for bed in bed_list:
    # print(bed)
    res = os.path.join(result_dir, os.path.basename(bed).replace(".bed",".fa"))
    os.system("bedtools getfasta -tab -fi %s -bed %s -fo %s -name" % (genome, bed, res))

66


In [15]:
gene_dir="/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/corr_ASE_m6A/oppo/for_experiment/overlap/"
fa_dir="/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/corr_ASE_m6A/oppo/for_experiment/fasta/"
gene_list = glob.glob("%s/*.bed" % gene_dir)
result_dir = "final_format/"
for gene in gene_list:
    res = os.path.join(result_dir, os.path.basename(gene).split(".")[0] + ".txt")
    fa = os.path.join(fa_dir, os.path.basename(gene).replace(".bed",'.fa'))
    df_gene = pd.read_table(gene, sep="\t", header=None)
    df_gene.columns = ['chr','s','e','snp','g_chr','g_s','g_e','g_n','d','str','type','ens']
    df_gene['term'] = df_gene['chr']+":"+df_gene['s'].astype(str)+":"+df_gene['snp']
    df_fa = pd.read_table(fa, sep="\t", header=None)
    df_fa.columns = ['term','fa']
    df = df_gene.merge(df_fa, on="term", how="left").dropna()
    # print(df.head())
    df['SNP_chr'],df['SNP_position'] = df['chr'],df['s']
    df['reference > alterative'] = df['snp'].str.split(":").str[4]
    df['RS ID'] = df['snp'].str.split(":").str[5]
    df['m6A_type'] = df['snp'].str.split(":").str[0]
    df['m6A_difference'] = df['snp'].str.split(":").str[1]
    df['expression_type'] = df['snp'].str.split(":").str[2]
    df['expression_difference'] = df['snp'].str.split(":").str[3]
    df['Gene Name'],df['Gene ensemble'] = df['g_n'], df['ens']
    df['Gene Start'], df['Gene End'], df['Gene Strand'] = df['g_s'], df['g_e'], df['str']
    df['sequence_surrounding_30nt'] = df['fa']
    selected_cols = ['SNP_chr','SNP_position','reference > alterative','RS ID', 
                     'm6A_type','m6A_difference','expression_type','expression_difference', 
                     'Gene Name','Gene ensemble','Gene Start','Gene End','Gene Strand','sequence_surrounding_30nt']
    df[selected_cols].to_csv(res, sep="\t", index=False)

In [16]:
# overlap with m6AVar db
db="/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/readCount/Fisher_test/sig/bed/overlap_withDB/m6AVar_hg19.txt"
term_list = pd.read_table(db,header=None).iloc[:,0].tolist()

result_dir = "/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/corr_ASE_m6A/oppo/for_experiment/final_format/overlap_m6AVar/"

txt_dir = "/home/galaxy/project/alleleSpecific_analysis/results/ASE_analysis/corr_ASE_m6A/oppo/for_experiment/final_format/"
txt_list = glob.glob("%s/*.txt" % txt_dir)
for txt in txt_list:
    df = pd.read_table(txt)
    df['tmp'] = df['SNP_chr'] + ":" + df['SNP_position'].astype(str)
    df['overlap_m6AVar'] = np.where(df['tmp'].isin(term_list), "Yes", "No")
    del df['tmp']
    result_file = os.path.join(result_dir, os.path.basename(txt))
    df.to_csv(result_file, sep="\t", index=False)